## Query 6

> Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos.

- Los top 10 son respecto a la cantidad
- Ordenes que NO tengan el producto mas allá de si tienen o no detalles en las ordenes

1. Importamos el dataset de detalles de ordenes (.data/raw/order_items.csv)
2. Filtramos registros con datos invalidos (product_id, quantity)
3. Mapeamos a (product_id, quantity)
4. Agrupamos por product_id y sumamos las cantidades
5. Nos quedamos los 10 más vendidos y extraemos los product_id
6. Broadcasteamos los product_id de los top 10
7. Importamos el dataset de ordenes (.data/raw/orders.csv)
8. Filtramos registros con datos invalidos (order_id)
9. Joineamos con los detalles de ordenes (order_id) con un left outer join
10. Agrupamos por order_id y sumamos la cantidad de productos que son de los top 10
11. Filtramos las ordenes que no tienen productos de los top 10
12. Contamos la cantidad de ordenes que cumplen la condición

In [23]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Query6_OrdenesSinProductosPopulares").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [24]:
# 1) Top 10 productos más vendidos por cantidad
order_items_df = spark.read.csv(
    "../../.data/raw/order_items.csv", header=True, inferSchema=True
)
oi_rdd = order_items_df.rdd

order_items_cleaned = oi_rdd.filter(
    lambda r: getattr(r, "product_id", None) is not None
    and getattr(r, "quantity", None) is not None
).map(
    lambda r: (
        str(getattr(r, "product_id", "")).strip(),
        int(getattr(r, "quantity", 0) or 0),
    )
)

qty_by_product = order_items_cleaned.reduceByKey(lambda a, b: a + b)

top10_products = qty_by_product.takeOrdered(10, key=lambda kv: -kv[1])
top10_set = set(pid for pid, _ in top10_products)
top10_bc = spark.sparkContext.broadcast(top10_set)

In [25]:
orders_df = spark.read.csv("../../.data/raw/orders.csv", header=True, inferSchema=True)
orders_rdd = orders_df.rdd

orders_min = orders_rdd.filter(lambda r: getattr(r, "order_id", None) is not None).map(
    lambda r: {"order_id": str(getattr(r, "order_id", "")).strip()}
)

order_items_per_order = (
    oi_rdd.filter(
        lambda r: getattr(r, "order_id", None) is not None
        and getattr(r, "product_id", None) is not None
    )
    .map(
        lambda r: (
            str(getattr(r, "order_id", "")).strip(),
            1 if str(getattr(r, "product_id", "")).strip() in top10_bc.value else 0,
        )
    )
    .reduceByKey(lambda a, b: a + b) # (order_id, count_of_top10_products_in_order)
).cache()  # este cache lo agrego ya que lo reutilizo cuando hago la query por segunda vez

orders_kv = orders_min.map(lambda x: (x["order_id"], 0))  # add it to allow the join

left_join = orders_kv.leftOuterJoin(
    order_items_per_order
)  # (order_id, (0, count_of_top10_products_in_order))

orders_without_top10 = left_join.filter(lambda kv: (kv[1][1] or 0) == 0)

count_orders_without_top10 = orders_without_top10.count()

In [26]:
print(f"Cantidad de órdenes sin productos del top 10: {count_orders_without_top10}")

Cantidad de órdenes sin productos del top 10: 4700000


Repito pero sin joinear con ordenes ya que no parece estar habiendo matchs entre `orders` y `order_items`.

In [27]:
orders_without_top10_without_join = order_items_per_order.filter(lambda kv: kv[1] == 0).count()

print(f"Cantidad de órdenes sin productos del top 10 (sin join): {orders_without_top10_without_join}")

Cantidad de órdenes sin productos del top 10 (sin join): 99507
